<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Step_back_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following this blogpost on how to explore step-back prompting.
https://levelup.gitconnected.com/step-back-prompting-how-to-make-llm-chatbots-smarter-than-ever-1b938673b8a2

In [ ]:
!pip install -qU langchain
!pip install -q fireworks-ai duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

In [ ]:
import os
from langchain.chat_models import ChatFireworks


os.environ["FIREWORKS_API_KEY"] = env['fireworks.ai']['apiKey']

llm = ChatFireworks(model="accounts/fireworks/models/mistral-7b-instruct-4k",
                        model_kwargs={"temperature": 0.1, "max_tokens": 150, "top_p": 0.9})

In [ ]:
import langchain
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

In [ ]:
# Testing RAG Fusion
llm.predict(""
"""<s>[INST] You are a helpful assistant that generates multiple search queries based on a single input query.

Generate multiple search queries related to: was chatgpt around while trump was president?

OUTPUT (4 queries): [/INST]""")

[llm/start] [1:llm:ChatFireworks] Entering LLM run with input:
{
  "prompts": [
    "Human: <s>[INST] You are a helpful assistant that generates multiple search queries based on a single input query.\n\nGenerate multiple search queries related to: was chatgpt around while trump was president?\n\nOUTPUT (4 queries): [/INST]"
  ]
}
[llm/end] [1:llm:ChatFireworks] [1.02s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. \"ChatGPT during Trump presidency\"\n2. \"ChatGPT and Trump administration\"\n3. \"ChatGPT and Trump policies\"\n4. \"ChatGPT and Trump controversies\"",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "1. \"ChatGPT during Trump presidency\

'1. "ChatGPT during Trump presidency"\n2. "ChatGPT and Trump administration"\n3. "ChatGPT and Trump policies"\n4. "ChatGPT and Trump controversies"'

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("Question", "{input}"),
        ("Assistant", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


ABSTRACT_QUESTION_GENERATOR_PROMPT="""<s>[INST] You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. You can use the examples below wrap by `>>>` as guidance on what are good paraphrase questions

>>>
Here are a few examples:
Question: Could the members of The Police perform lawful arrests?
Assistant: what can the members of The Police do?

Question: Jan Sindel’s was born in what country?
Assistant:  what is Jan Sindel’s personal history?
>>>

Question: {question} [/INST]
"""


prompt = PromptTemplate(input_variables=['language', 'topic'], template=ABSTRACT_QUESTION_GENERATOR_PROMPT)

chain = LLMChain(llm=llm, prompt=prompt)

We are not using chat `ChatPromptTemplate` here as we are not chatting to chatbot, we are using LLM not to answer a question but rather to do a task to us.

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate


# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = PromptTemplate(input_variables=["input", "ouput"], template="Question: {input}\n Answer:{output}")

few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="", # this is required
    input_variables=[] # this is required
)


ABSTRACT_QUESTION_GENERATOR_PROMPT="""<s>[INST] You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. You can use the examples below wrap by `>>>` as guidance on what are good paraphrase questions

>>>
Here are a few examples:
{few_shot_prompt}
>>>

Question: {question} [/INST]"""


prompt = PromptTemplate.from_template(ABSTRACT_QUESTION_GENERATOR_PROMPT)

In [ ]:
chain.run(question="was chatgpt around while trump was president?")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "was chatgpt around while trump was president?"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatFireworks] Entering LLM run with input:
{
  "prompts": [
    "Human: <s>[INST] You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. You can use the examples below wrap by `>>>` as guidance on what are good paraphrase questions\n \n>>>\nHere are a few examples:\nQuestion: Could the members of The Police perform lawful arrests?\nAssistant: what can the members of The Police do?\n\nQuestion: Jan Sindel’s was born in what country?\nAssistant:  what is Jan Sindel’s personal history?\n>>>\n\nQuestion: was chatgpt around while trump was president? [/INST]"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatFireworks] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "What is the historical contex

'What is the historical context of the time period when ChatGPT was developed and when Trump was president?'

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.schema.runnable import RunnableLambda

# function that will allow us to run this to a langchain expression
def query_lambda(input):
    # Perform some operations on the input
    return input

# Create a RunnableLambda component with your custom function
custom_component = RunnableLambda(query_lambda)

query_parser =  custom_component | CommaSeparatedListOutputParser()

In [ ]:
import time
from langchain.utilities import DuckDuckGoSearchAPIWrapper

search = DuckDuckGoSearchAPIWrapper(max_results=4)

def retriever_list(query):
    answer = ''
    ques = ''
    for question in query:
        ques += question
        ques += '/'
        if question[-1] == '?':
            ans = search.run(ques)
            ques = ''
            answer += ans
            time.sleep(2)
    print("Answer: ", answer)
    return answer

static_query_reteriever = query_parser | retriever_list

static_query_reteriever.invoke("What is the historical context of the time period when ChatGPT was developed and when Trump was president?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the historical context of the time period when ChatGPT was developed and when Trump was president?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:query_lambda] Entering Chain run with input:
{
  "input": "What is the historical context of the time period when ChatGPT was developed and when Trump was president?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:query_lambda] s] Exiting Chain run with output:
{
  "output": "What is the historical context of the time period when ChatGPT was developed and when Trump was president?"
}
[chain/start] [1:chain:RunnableSequence > 3:parser:CommaSeparatedListOutputParser] Entering Parser run with input:
{
  "input": "What is the historical context of the time period when ChatGPT was developed and when Trump was president?"
}
[chain/end] [1:chain:RunnableSequence > 3:parser:CommaSeparatedListOutputParser] s] Exiting Parser run with output:
{
  "outp

'Aleksandra Yosifova 14 Aug 2023 7 min read ChatGPT isn\'t the first language model; it isn\'t even the first GPT model. But it made a significant leap in natural language processing—popularizing large language models and accelerating the adoption of AI. What factors contributed to ChatGPT\'s success? ChatGPT is an artificial intelligence (AI) chatbot built on top of OpenAI\'s foundational large language models (LLMs) like GPT-4 and its predecessors. This chatbot has redefined the standards of... When OpenAI launched ChatGPT, with zero fanfare, in late November 2022, the San Francisco-based artificial-intelligence company had few expectations. Certainly, nobody inside OpenAI was prepared... ChatGPT, which stands for "Chat Generative Pre-training Transformer," is a large language model developed by OpenAI. It is based on the GPT (Generative Pre-training Transformer) architecture,...'